In [1]:
import requests
import json
import pandas as pd

In [19]:
# Assume the JSON data is stored in a variable called `json_data`
data = json.loads(response.text)

In [22]:
# Extract the reviews from the JSON data and convert it to a DataFrame
reviews = pd.DataFrame(data['reviews'])

In [25]:
reviews.to_csv("reviews_sample.csv",index=False)

In [3]:
asin_list_path = '/Users/vladbordei/Documents/Development/ProductExplorer/data/external/asin_list.csv'
asin_list = pd.read_csv(asin_list_path)['asin'].tolist()

In [4]:
asin_list

['B08X2324ZL',
 'B09MQ689XL',
 'B0BCW8X98V',
 'B09NR5JW8Y',
 'B0BHS5VPCZ',
 'B0BCVY1DHR',
 'B0BNXCJ3MV',
 'B0BCTTCSBZ',
 'B091325ZMB',
 'B093HCXGC9',
 'B0BL6MR1KN',
 'B09PN11Y65',
 'B0BLNBS36G',
 'B0BVZ3H4MB',
 'B0BW8Y2B8Z',
 'B0BDFZPNVY']

In [ ]:
# Loop over ASINs
for asin in asin_list:
    details = get_product_details(asin)
    reviews = get_product_reviews(asin)
    update_firestore(asin, details, reviews)

In [13]:
# old api

import time
import asyncio
import nest_asyncio
import aiohttp
import json
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore






# Your ASINs
asin_list = ['B08X2324ZL', 'B09MQ689XL']

# Amazon Scraper details
base_url = "https://asin-data.p.rapidapi.com/request"
api_key = "70201ee0c8ed29661bc6ae00a84341fb"
headers = {
	"X-RapidAPI-Key": "4da31a08e5mshaca05d98a3d9d6ep1fffb1jsn019717508cc8",
	"X-RapidAPI-Host": "asin-data.p.rapidapi.com"
}

# Firestore details
cred_path = '/Users/vladbordei/Documents/Development/ProductExplorer/notebooks/productexplorerdata-firebase-adminsdk-ulb3d-465f23dff3.json'

# Initialize Firestore
cred = credentials.Certificate(cred_path)
if not firebase_admin._apps:
    firebase_admin.initialize_app(cred)
db = firestore.client()

async def get_product_details(asin):
    start = time.time()
    async with aiohttp.ClientSession() as session:
        url = f"{base_url}{asin}"
        async with session.get(url, headers=headers, params={"api_key": api_key}) as response:
            if response.status != 200:
                print(f"Failed to fetch product details for {asin}. HTTP status: {response.status}")
                return None
            print(f"Fetching product details for {asin} took {time.time() - start} seconds.")
            return await response.json()

async def fetch(session, page_var, asin):
    params = {
        "type":"reviews",
        "asin":asin,
        "amazon_domain":"amazon.com",
        "reviewer_type":"verified_purchase",
        "sort_by":"most_recent",
        "global_reviews":"false",
        "page":page_var
    }
    async with session.get(url=base_url, headers=headers, params=params) as response:
        if response.status != 200:
            print(f"Failed to fetch reviews for {asin} page {page_var}. HTTP status: {response.status}")
            return None
        return await response.json()

async def get_product_reviews(asin):
    start = time.time()
    async with aiohttp.ClientSession() as session:
        pages = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10"]
        tasks = [fetch(session, page_var, asin) for page_var in pages]
        results = await asyncio.gather(*tasks)
        print(f"Fetching product reviews for {asin} took {time.time() - start} seconds.")
        return results

def update_firestore(asin, details, reviews):
    if details is None or reviews is None or any(review is None for review in reviews):
        print(f"Skipping Firestore update for {asin} due to missing data.")
        return
    start = time.time()
    doc_ref = db.collection('products').document(asin)
    doc_ref.set({
        'details': details,
        'reviews': reviews
    }, merge=True)
    print(f"Updating Firestore for {asin} took {time.time() - start} seconds.")

async def process_asin(asin):
    details = await get_product_details(asin)
    if details is None:
        print(f"Skipping {asin} due to failed details fetch.")
        return
    reviews = await get_product_reviews(asin)
    if reviews is None or any(review is None for review in reviews):
        print(f"Skipping {asin} due to failed reviews fetch.")
        return
    update_firestore(asin, details, reviews)

async def main():
    tasks = [process_asin(asin) for asin in asin_list]
    await asyncio.gather(*tasks)

loop = asyncio.get_event_loop()
loop.run_until_complete(main())


Failed to fetch product details for B08X2324ZL. HTTP status: 404
Skipping B08X2324ZL due to failed details fetch.
Failed to fetch product details for B09MQ689XL. HTTP status: 404
Skipping B09MQ689XL due to failed details fetch.


In [19]:
# New api

import time
import asyncio
import aiohttp
import json
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

# Your ASINs
asin_list = ['B08X2324ZL', 'B09MQ689XL']

# Amazon Scraper details
details_base_url = "https://amazon23.p.rapidapi.com/product-details"
reviews_base_url = "https://amazon23.p.rapidapi.com/reviews"
headers = {
    "X-RapidAPI-Key": "4da31a08e5mshaca05d98a3d9d6ep1fffb1jsn019717508cc8",
    "X-RapidAPI-Host": "amazon23.p.rapidapi.com"
}

# Firestore details
cred_path = '/Users/vladbordei/Documents/Development/ProductExplorer/notebooks/productexplorerdata-firebase-adminsdk-ulb3d-465f23dff3.json'

# Initialize Firestore
cred = credentials.Certificate(cred_path)
if not firebase_admin._apps:
    firebase_admin.initialize_app(cred)
db = firestore.client()

async def get_product_details(asin):
    start = time.time()
    async with aiohttp.ClientSession() as session:
        params = {"asin": asin, "country": "US"}
        async with session.get(details_base_url, headers=headers, params=params) as response:
            if response.status != 200:
                print(f"Failed to fetch product details for {asin}. HTTP status: {response.status}")
                return None
            print(f"Fetching product details for {asin} took {time.time() - start} seconds.")
            return await response.json()

async def fetch(session, page_var, asin):
    params = {
        "asin": asin,
        "sort_by": "recent",
        "page": page_var,
        "country": "US"
    }
    async with session.get(reviews_base_url, headers=headers, params=params) as response:
        if response.status != 200:
            print(f"Failed to fetch reviews for {asin} page {page_var}. HTTP status: {response.status}")
            return None
        return await response.json()

async def get_product_reviews(asin):
    start = time.time()
    async with aiohttp.ClientSession() as session:
        pages = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10"]
        tasks = [fetch(session, page_var, asin) for page_var in pages]
        results = await asyncio.gather(*tasks)
        print(f"Fetching product reviews for {asin} took {time.time() - start} seconds.")
        return results

def update_firestore(asin, details, reviews):
    if details is None or reviews is None or any(review is None for review in reviews):
        print(f"Skipping Firestore update for {asin} due to missing data.")
        return
    start = time.time()
    doc_ref = db.collection('products').document(asin)
    doc_ref.set({
        'details': details,
        'reviews': reviews
    }, merge=True)
    print(f"Updating Firestore for {asin} took {time.time() - start} seconds.")

async def process_asin(asin):
    details = await get_product_details(asin)
    if details is None:
        print(f"Skipping {asin} due to failed details fetch.")
        return
    reviews = await get_product_reviews(asin)
    if reviews is None or any(review is None for review in reviews):
        print(f"Skipping {asin} due to failed reviews fetch.")
        return
    update_firestore(asin, details, reviews)

async def main():
    tasks = [process_asin(asin) for asin in asin_list]
    await asyncio.gather(*tasks)

loop = asyncio.get_event_loop()
loop.run_until_complete(main())


Fetching product details for B08X2324ZL took 2.5519559383392334 seconds.
Fetching product reviews for B08X2324ZL took 2.552313804626465 seconds.
Updating Firestore for B08X2324ZL took 0.5691540241241455 seconds.
Fetching product details for B09MQ689XL took 5.8336639404296875 seconds.
Fetching product reviews for B09MQ689XL took 2.2384912967681885 seconds.
Updating Firestore for B09MQ689XL took 0.5269620418548584 seconds.


In [24]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

# Firestore details
cred_path = '/Users/vladbordei/Documents/Development/ProductExplorer/notebooks/productexplorerdata-firebase-adminsdk-ulb3d-465f23dff3.json'

# Initialize Firestore
cred = credentials.Certificate(cred_path)
if not firebase_admin._apps:
    firebase_admin.initialize_app(cred)
db = firestore.client()

# ASIN
asin = 'B09MQ689XL'  # Replace with the ASIN you are interested in

# Get the document with the provided ASIN
doc_ref = db.collection('products').document(asin)

# Get the document data
doc = doc_ref.get()

# Check if document exists
if doc.exists:
    # Get reviews
    reviews = doc.to_dict().get('reviews', {})
    print(reviews)
else:
    print(f"No such document for ASIN: {asin}")


[{'total_reviews': 2379, 'result': [{'review': 'It is good for the price. Next time I will pull the trigger for the better brand.', 'verified_purchase': True, 'date': {'unix': 1690164000, 'date': 'July 24, 2023'}, 'asin': {'variant': 'B079R764PP', 'original': 'B09MQ689XL'}, 'id': 'R1BQI18TCROVNH', 'review_data': 'Reviewed in the United States 🇺🇸 on July 24, 2023', 'rating': 3, 'title': '3.0 out of 5 starsNot Bad for the Price', 'name': 'Amazon Customer'}, {'review': 'It was easy to install perfect', 'verified_purchase': True, 'date': {'unix': 1690077600, 'date': 'July 23, 2023'}, 'asin': {'variant': 'B09MQ689XL', 'original': 'B09MQ689XL'}, 'id': 'R3K9S8MFVIOFFS', 'review_data': 'Reviewed in the United States 🇺🇸 on July 23, 2023', 'rating': 5, 'title': '5.0 out of 5 starsPerfect', 'name': 'Sherry'}, {'review': 'I went for a large rectangular structure, something like 4x10.the issue is the 10ft long sides have bowed out from the weight of the material i put inside.I saw others with like 

# Product Data


In [27]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

# Firestore details
cred_path = '/Users/vladbordei/Documents/Development/ProductExplorer/notebooks/productexplorerdata-firebase-adminsdk-ulb3d-465f23dff3.json'

# Initialize Firestore
cred = credentials.Certificate(cred_path)
if not firebase_admin._apps:
    firebase_admin.initialize_app(cred)
db = firestore.client()

# ASIN
asin = 'B09MQ689XL'  # Replace with the ASIN you are interested in

# Get the document with the provided ASIN
doc_ref = db.collection('products').document(asin)

# Get the document data
doc = doc_ref.get()

# Check if document exists
if doc.exists:
    # Get reviews
    product_details = doc.to_dict().get('details', {})
    print(product_details)
else:
    print(f"No such document for ASIN: {asin}")


{'result': [{'asin': 'B09MQ689XL', 'other_sellers': [{'seller': ' Overwhelming ', 'position': 1, 'url': 'https://www.amazon.comundefined', 'price': {'current_price': None, 'symbol': '$', 'currency': 'USD'}}], 'delivery_message': '', 'sponsored_products': [], 'bestsellers_rank': [], 'categories': [{'url': 'https://www.amazon.com/Patio-Lawn-Garden/b/ref=dp_bc_aui_C_1/140-9313597-5423960?ie=UTF8&node=2972638011', 'category': 'Patio, Lawn & Garden'}, {'url': 'https://www.amazon.com/Plants-Seeds-Flowers-Outdoor-Living/b/ref=dp_bc_aui_C_2/140-9313597-5423960?ie=UTF8&node=3610851', 'category': 'Gardening & Lawn Care'}, {'url': 'https://www.amazon.com/b/ref=dp_bc_aui_C_3/140-9313597-5423960?ie=UTF8&node=3480694011', 'category': 'Pots, Planters & Container Accessories'}, {'url': 'https://www.amazon.com/Planter-Raised-Beds/b/ref=dp_bc_aui_C_4/140-9313597-5423960?ie=UTF8&node=14623212011', 'category': 'Raised Beds'}], 'url': 'https://www.amazon.com/dp/B09MQ689XL', 'price': {'before_price': 169.99

In [28]:
product_details

{'result': [{'asin': 'B09MQ689XL',
   'other_sellers': [{'seller': ' Overwhelming ',
     'position': 1,
     'url': 'https://www.amazon.comundefined',
     'price': {'current_price': None, 'symbol': '$', 'currency': 'USD'}}],
   'delivery_message': '',
   'sponsored_products': [],
   'bestsellers_rank': [],
   'categories': [{'url': 'https://www.amazon.com/Patio-Lawn-Garden/b/ref=dp_bc_aui_C_1/140-9313597-5423960?ie=UTF8&node=2972638011',
     'category': 'Patio, Lawn & Garden'},
    {'url': 'https://www.amazon.com/Plants-Seeds-Flowers-Outdoor-Living/b/ref=dp_bc_aui_C_2/140-9313597-5423960?ie=UTF8&node=3610851',
     'category': 'Gardening & Lawn Care'},
    {'url': 'https://www.amazon.com/b/ref=dp_bc_aui_C_3/140-9313597-5423960?ie=UTF8&node=3480694011',
     'category': 'Pots, Planters & Container Accessories'},
    {'url': 'https://www.amazon.com/Planter-Raised-Beds/b/ref=dp_bc_aui_C_4/140-9313597-5423960?ie=UTF8&node=14623212011',
     'category': 'Raised Beds'}],
   'url': 'https:

In [29]:
# Ramas to do: de stabilizat la un singur API call, altfel facut un JSON dump cu ce date sunt primite. Otherwise, este foarte dificil sa faci debugging.

In [ ]:
# Este posbil sa fie rescrise cele 10 reviews again and again, in loc sa fie toate notate in baza de date pentru un ASIN.